# [CPSC 310](https://github.com/GonzagaCPSC310) Data Mining
[Gonzaga University](https://www.gonzaga.edu/)

[Gina Sprint](http://cs.gonzaga.edu/faculty/sprint/)

# Amazon SageMaker
What are our learning objectives for this lesson?
* Make an Amazon Web Services (AWS) account
* Set up a Jupyter Notebook instance on Amazon SageMaker
* Perform simple k-means clustering using SageMaker

Content used in this lesson is based upon information in the following sources:
* [Amazon SageMaker developer guide](https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html)

## Warm-up Task(s)
1. Make a [free AWS account](https://aws.amazon.com/free/?awsf.Free%20Tier%20Types=categories%23featured) if you don't already have one
    * Later, you can get free AWS credits as a student through AWS Educate: https://aws.amazon.com/education/awseducate/ 
1. Go to console.aws.amazon.com and sign in to your account
1. Make sure you have ClusteringFun/shirt_sizes.csv downloaded on your machine somewhere

## Helpful Links
* [Amazon SageMaker developer guide](https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html)
* [Amazon E2 instance types](https://aws.amazon.com/ec2/instance-types/) and more specifically, [Amazon SageMaker ML instance types](https://aws.amazon.com/sagemaker/pricing/instance-types/)

## Tips on Viewing the Algorithm Parameters
This would be if you wanted to "view your model." For k-means, this would be seeing the cluster centroids. This takes more work than I think it should because the `sagemaker.KMeans` class does not expose the centroids, it instead writes them to the model file, model.tar.gz

* You'll need to create a training job using the low-level SDK for Python so you have the job_name in your code. See this part of the developer guide for how to do this: https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-train-model-create-training-job.html
    * I'll note that you could manually go to your S3 bucket and find the job name that was automatically used via `kmeans.fit(kmeans.record_set(train_set[0]))` of the high-level Python library. It will be the name of the folder storing a model.tar.gz file, something like `kmeans-2019-04-29-02-11-06-721`
* You'll need to load the trained model via https://aws.amazon.com/blogs/machine-learning/analyze-us-census-data-for-population-segmentation-using-amazon-sagemaker/